In [ ]:
import psycopg2
import pandas as pd

DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432




connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)
#кол-во вакансий
query3_1 = f'''select count(name) 
           from VACANCIES 
           
        '''
# кол-во рабодателей        
query3_2 = f'''select  name
           from EMPLOYERS
           
       '''
# кол-во регионов
query3_3 = f'''select count(id)
           from AREAS
           
        '''  
# кол-во сфер деятельности       
query3_4 = f'''select count(id)
          # from INDUSTRIES
           
       # '''    
        
 
               
            
df = pd.read_sql_query(query3_4, connection)
df






/var/folders/pr/r8xvpl013ml0gp82c2nfh20h0000gn/T/ipykernel_78237/2219897120.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,294


In [ ]:
# запрос который показывает 5 городов по кол-во вакансий

quer4_1 = f'''select a.name, count(v.id)
           from VACANCIES v
           JOIN AREAS a ON a.id = v.area_id
           GROUP BY 1
           ORDER BY 2 DESC
           LIMIT 5
           
        '''  
# кол-во вакансий где хотя бы указана зп в одном пункте        
query4_2 = f'''select count(id)
           from VACANCIES 
           WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
           
           
        '''   
 # средняя значения нижней и верней границы зп вилки       
query4_3 = f'''select 
             ROUND(AVG(salary_from)) AS avg1,
             ROUND(AVG(salary_to)) AS avg2
           from VACANCIES 
           
        '''  
  #  кол-во вакансий от типа работы и графика         
query4_4 = f'''select schedule,
             employment , count(id)
           from VACANCIES 
           GROUP BY 1,2
           ORDER BY 3 DESC
           
        '''  
  #  кол-во вакансий с опытом работы      
query4_5 = f'''select experience, count(id)
           from VACANCIES 
           GROUP BY 1
           ORDER BY 2 
           
        '''                              
         

df = pd.read_sql_query(query4_5, connection)
df        

# Москва и Санкт_Петербург городами где много вакансий
# Работодатели в почти половине случаев указывают зп
# самым популярный способом работы является полный день - полная занятность после чего на 2 место идет удаленка
# от 1 до 3 лет самое распространенный срок работы 

/var/folders/pr/r8xvpl013ml0gp82c2nfh20h0000gn/T/ipykernel_78237/957839048.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query5, connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


In [134]:
# топ 5 работодателей по кол-во вакансий 
query5_1 = f'''select e.name, count(v.id)
           from VACANCIES v
           JOIN EMPLOYERS e ON e.id = v.employer_id
           GROUP BY 1
           ORDER BY 2 DESC
           LIMIT 5
           
        '''    
        # регион где наибольшее кол-во работодателей , но где нет вакансий 
query5_2 = f'''SELECT 
    a.name as "Название региона",
    COUNT(distinct e.id) as "Количество работодателей",
    COUNT(distinct v.id) as "Количество вакансий"
FROM   
    AREAS a
LEFT JOIN EMPLOYERS e ON e.area = a.id
LEFT JOIN VACANCIES v ON v.area_id = a.id
WHERE v.id is NULL
GROUP BY a.name
ORDER BY 2 DESC
LIMIT 1
'''     
# кол-во регионов где работодалей публикует вакансии 
query5_3 = f'''select e.name, count(distinct a.id)
           from VACANCIES v
           JOIN EMPLOYERS e ON e.id = v.employer_id
           JOIN AREAS a ON v.area_id = a.id
           
           GROUP BY 1
           ORDER BY 2 DESC
           
           
        '''    
# кол-во работодателей где не указана сфера деятетельности         
query5_4 = f'''select count(e.id)
               from EMPLOYERS e
               LEFT JOIN EMPLOYERS_INDUSTRIES ei ON e.id = ei.employer_id 
               LEFT JOIN INDUSTRIES i ON i.id = ei.industry_id
               WHERE i.name IS NULL
           
        '''    
 #  работодатели где указано 4 сферы деятельности        
query5_5 = f'''select e.name, count(i.id)
               from EMPLOYERS e
               LEFT JOIN EMPLOYERS_INDUSTRIES ei ON e.id = ei.employer_id 
               LEFT JOIN INDUSTRIES i ON i.id = ei.industry_id
               GROUP BY 1
               HAVING count(i.id) = 4
               ORDER BY 1
               
        '''    
  # кол-во работодатели где указано разбработка программного обеспечения       
query5_6 = f'''select count(e.id)
               from EMPLOYERS e
               LEFT JOIN EMPLOYERS_INDUSTRIES ei ON e.id = ei.employer_id 
               LEFT JOIN INDUSTRIES i ON i.id = ei.industry_id
               WHERE i.name = 'Разработка программного обеспечения'
             '''   
             
import requests
from bs4 import BeautifulSoup

url='https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response=requests.get(url)
page=BeautifulSoup(response.text,'html.parser')

cities_here=str(page.find('table', class_="standard"))
cities_here=cities_here.split('title="')[3:]
cities=[]
for i in cities_here:
    cities.append(i.split('">')[0])     
                                                             


query_5_7=f'''
select
     a.name as name,
     count(v.id) cnt
from vacancies v
left join employers e on e.id=v.employer_id
left join areas a on a.id=v.area_id
where e.name='Яндекс'
group by 1
having a.name in {tuple(cities)}
union all
select
     'total',
     sum(cnt)
from 
     (select
          a.name as name,
          count(v.id) cnt
     from vacancies v
     left join employers e on e.id=v.employer_id
     left join areas a on a.id=v.area_id
     where e.name='Яндекс'
     group by 1
     having a.name in {tuple(cities)})subquery
'''


  

df = pd.read_sql_query(query5_5, connection)
df   

/var/folders/pr/r8xvpl013ml0gp82c2nfh20h0000gn/T/ipykernel_78237/1960021748.py:105: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query5_5, connection)


,name,count
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4
...,...,...
1131,ЮРИОН,4
1132,ЮТИП Технологии,4
1133,ЯКласс,4
1134,ЯрНео,4


In [ ]:
# кол-во вакансий имеющий отношение к данным
query6_1 = f'''select count(id)
               from VACANCIES
               where lower(name) LIKE '%data%' OR lower(name) LIKE '%данн%' 
             ''' 
             
# кол-во вакансий для дата сайентиста  без опыта             
query6_2 = f'''select
    count(v.name)

from vacancies v

WHERE
(
    (name ILIKE '%data scientist%') OR (name ILIKE '%data science%') OR (name ILIKE '%исследователь данных%')
    OR ((name ILIKE '%ML%') AND (name NOT ILIKE '%HTML%'))
    OR (name ILIKE '%machine learning%')
    OR (name ILIKE '%машинн%обучен%')
    AND (key_skills IS NOT NULL)
)

    AND (v.name ILIKE '%junior%'
    OR v.experience ILIKE '%Нет опыта%'
    OR v.employment ILIKE '%Стажировка%')
             '''    
   # кол-во вакансий которые еще указаны sql           
query6_3 = f''' select
 
    count(id)
    

from vacancies v

WHERE
(
    ((name ILIKE '%data scientist%') OR (name ILIKE '%data science%') OR (name ILIKE '%исследователь данных%')
    OR ((name ILIKE '%ML%') AND (name NOT ILIKE '%HTML%'))
    OR (name ILIKE '%machine learning%')
    OR (name ILIKE '%машинн%обучен%'))
    AND (key_skills like '%postgres%' or key_skills like '%SQL%'))


             '''   
             
query6_4 = f''' select
 
    count(id)
    

from vacancies v

WHERE
(
    ((name ILIKE '%data scientist%') OR (name ILIKE '%data science%') OR (name ILIKE '%исследователь данных%')
    OR ((name ILIKE '%ML%') AND (name NOT ILIKE '%HTML%'))
    OR (name ILIKE '%machine learning%')
    OR (name ILIKE '%машинн%обучен%'))
    AND (key_skills like '%Python%'))


             '''  
query6_5=f'''SELECT 
                    ROUND(AVG(
                        LENGTH(v.key_skills) - LENGTH(REPLACE(v.key_skills, CHR(9), '')) + 1
                        ), 2) AS "Среднее количество навыков"
                FROM public.vacancies v
                WHERE 
                    (v.name ILIKE '%data scientist%' OR 
                    v.name ILIKE '%data science%' OR
                    v.name ILIKE '%исследователь данных%' OR
                    (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
                    v.name ILIKE '%machine learning%' OR
                    v.name ILIKE '%машинн%обучен%%') 

'''  

query6_6 = f'''        
     select experience,
       round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)), 0) avg_sal
       from public.vacancies
       where (name ilike '%data scientist%'
       or name ilike '%data science%'
       or name ilike '%исследователь данных%'
       or (name like '%ML%' and name not ilike '%HTML%')
       or name ilike '%machine learning%' or name ilike '%машинн%обучен%'
       )and  (salary_from is not null or salary_to is not null)
    group by 1
              
   '''          
 
df = pd.read_sql_query(query6_6, connection)
df  
# лишь 2,88% вакансий рассматриваеются на стажировку
#  в 64 % знание python вместо sql упрощает поиск работы
# в среднем ключевых навыков указывают 6-7
# без опыта работы можно получить в среднем 74643 рублей

           

/var/folders/pr/r8xvpl013ml0gp82c2nfh20h0000gn/T/ipykernel_78237/4258781309.py:91: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query6_6, connection)


,experience,avg_sal
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


In [ ]:
query7_1 = f'''        
     select distinct name , count(id),
     FROM VACANCIES 
     GROUP BY 1
     ORDER BY 2 DESC
     LIMIT 5
       
              
   '''   
   # самые популярные вакансии  это системный администратор , Программист 1С , Бизнес-аналитик, Специалист технической поддержки, Системный аналитик	     
 
query7_2 = f'''        
     select distinct   a.name, count(v.id)
     FROM VACANCIES v
     LEFT JOIN AREAS a ON a.id = v.area_id
     GROUP BY 1
     ORDER BY 2 DESC
     LIMIT 5
       
              
   ''' 
   # самые популярные города являются Москва, Санкт-Петербург , Минск , Новосибирск, Алматы
df = pd.read_sql_query(query7_2, connection)
df  

/var/folders/pr/r8xvpl013ml0gp82c2nfh20h0000gn/T/ipykernel_78237/22824131.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query7_2, connection)


,name,count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
